In [1]:
import json, math
import requests

In [ ]:
def count_freq(data):
    save = {}
    freq = []
    for i in tqdm(range(len(data))):
        wiki_title = data[i]['requested_rewrite']['subject']

        # wiki_title = "George Rankin"
        headers = {'User-Agent': '<YOUR HEADERS INFO AND EMAIL>'}
        url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{}/monthly/2021100100/2021103100'
        resp = requests.get(url.format(wiki_title), headers=headers)
        try:
            views = resp.json()['items'][0]['views']
        # print(views)
        except KeyError:
            print(wiki_title, resp.json())
            views = 1
        
        data[i]['subject_freq'] = views
        save[wiki_title] = views
        freq.append(math.log(views, 10))
        return freq

In [2]:
def get_id(name):
    params = {
            'action': 'wbsearchentities',
            'format': 'json',
            'search': name,
            'language': 'en',
            'type': 'item',
        }
    url = 'https://www.wikidata.org/w/api.php'


    # Construct the URL manually
    constructed_url = f"{url}?{'&'.join([f'{key}={value}' for key, value in params.items()])}"
    #print("Constructed URL:", constructed_url)
    headers ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
    try:
        # Use requests.get to fetch the response
        response = requests.get(constructed_url,headers=headers)

        # Check if the request was successful (status code 200)
        if response.ok:
            # Print the response content
            data = json.loads(response.text)
            id = data['search'][0]['id']
            return id
        else:
            # Print an error message if the request was not successful
            print(f"Request failed with status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [3]:
def count_edges_subject(subject):
  WDSPARQL_URL = 'https://query.wikidata.org/sparql' 
  wid = get_id(subject)
  query = f"""
  SELECT (COUNT(?neighbor) AS ?edgeCount)
  WHERE {{
  wd:{wid} ?p ?neighbor.
  }}
  """
  params = {
          'format': 'json',
          'query': query
      }
  # Construct the URL manually
  constructed_url = f"{WDSPARQL_URL}?{'&'.join([f'{key}={value}' for key, value in params.items()])}"
  #print("Constructed URL:", constructed_url)
  headers ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
  try:
      # Use requests.get to fetch the response
      response = requests.get(constructed_url,headers=headers)

      # Check if the request was successful (status code 200)
      if response.ok:
        data = response.json()
        ans = data['results']['bindings'][0]['edgeCount']['value']
        return ans
      else:
          # Print an error message if the request was not successful
          print(f"Request failed with status code: {response.status_code}")

  except requests.exceptions.RequestException as e:
      print(f"An error occurred: {e}")
      
def count_path_num(subject,object,subject_wd_id=None,object_wd_id=None):
    WDSPARQL_URL = 'https://query.wikidata.org/sparql' 
    if subject_wd_id == None:
        subject_wd_id = get_id(subject)
    if object_wd_id == None:
        object_wd_id = get_id(object)
    query = f"""
    SELECT (COUNT(DISTINCT ?middle) AS ?pathCount)
    WHERE {{
    wd:{subject_wd_id} ?p1 ?middle.
    ?middle ?p2 wd:{object_wd_id}.
    FILTER (?middle != wd:{subject_wd_id} && ?middle != wd:{object_wd_id})
    }}
    """
    query12 = f"""
    SELECT (COUNT(*) AS ?pathCount) WHERE {{
        {{
            wd:{subject_wd_id} ?p wd:{object_wd_id}.
        }}
        UNION
        {{
            wd:{subject_wd_id} ?p1 ?x.
            ?x ?p2 wd:{object_wd_id}.
        }}
        UNION
        {{
            wd:{subject_wd_id} ?p1 ?x.
            ?x ?p2 ?y.
            ?y ?p3 wd:{object_wd_id}.
        }}
    }}"""
    
    query1hop = f"""
    SELECT (COUNT(*) AS ?pathCount) WHERE {{
        {{
            wd:{subject_wd_id} ?p wd:{object_wd_id}.
        }}
        
    }}"""

    params = {
            'format': 'json',
            'query': query1hop
        }
    # Construct the URL manually
    constructed_url = f"{WDSPARQL_URL}?{'&'.join([f'{key}={value}' for key, value in params.items()])}"
    #print("Constructed URL:", constructed_url)
    headers ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
    try:
        # Use requests.get to fetch the response
        #response = requests.get(constructed_url,headers=headers)
        response = requests.get(WDSPARQL_URL, params=params, headers=headers)
        # Check if the request was successful (status code 200)
        if response.ok:
            data = response.json()
            return data['results']['bindings'][0]['pathCount']['value']
            
        else:
            # Print an error message if the request was not successful
            print(f"Request failed with status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")